# Adding Loss Function

In [30]:
import numpy as np

class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        # We want numpy to create a weights that will be the size of
        # number of inputs times the number of neurons we want

        # The parameters of randn is the shape,
        # in this case, randn(number of columns, number of rows)
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)

        # first parameter in np.zeros is the shape
        # Remember, each neuron has a unique bias, so that is how many
        # biases we will initially generate
        self.biases = np.zeros((1, n_neurons))
        # what randn does is just a Gaussian Distribution around 0

    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities
        
class Loss:
    def calculate(self, output, y):
        """
        
        :param output: Output from the model
        :param y: Intended target values
        :return: data loss
        """
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss
    
class Loss_CategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        """
        
        :param y_pred: Values from the NN
        :param y_true: Target training values
        :return: 
        """
        samples = len(y_pred)
        
        # Values must be between:
        # ---- Start: 16.11809565095832
        # ---- End : 1.0000000494736474e-07
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        # Add functionality to handle 'Scalar' or 'One Hot Encoded' values
        # - Scalar [  1,     0]
        # - OneHot [[0,1], [1,0]]
        
        # 1 - Dimension (Scalar)
        if len(y_true.shape) == 1:
            # y_pred_clipped[y_pred_clipped[0,1,2], [0, 1, 1]]
            correct_confidences = y_pred_clipped[range(samples), y_true]
            
        # 2 - Dimension (One Hot Encoded)
        elif len(y_true.shape) == 2:
            # 1. Multiply each vector
            #       Example: a[0.7, 0.1, 0.2] * b[1, 0, 0] = [[0.7 * 1, 0.1 * 0, ...]]
            # 2. Do sum along AXIS ONE (row)
            #       Example: [[0.7, 0.0, 0.0],....] = [[0.7+0.0+0.0],....]
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
            
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

In [31]:
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

# Create input/output data
X, y = spiral_data(samples=100, classes=3)

In [32]:
# (Dense) Layer - 1
dense1 = Layer_Dense(n_inputs=2, n_neurons=3)
activation1 = Activation_ReLU()

# (Dense) Layer - 2
dense2 = Layer_Dense(n_inputs=3, n_neurons=3)
activation2 = Activation_Softmax()

In [33]:
dense1.forward(X)
activation1.forward(dense1.output)

dense2.forward(activation1.output)
activation2.forward(dense2.output)

print(activation2.output[:5])

[[0.33333334 0.33333334 0.33333334]
 [0.33331734 0.3333183  0.33336434]
 [0.3332888  0.33329153 0.33341965]
 [0.33325943 0.33326396 0.33347666]
 [0.33323312 0.33323926 0.33352762]]


In [34]:
loss_function = Loss_CategoricalCrossEntropy()

loss = loss_function.calculate(activation2.output, y)
print("Loss: ", loss)

Loss:  1.098445
